In [207]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import time, timedelta
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import ftfy
import datetime

In [106]:
# API V1
# ['Cité des Congrès', 'STEREOLUX', 'Zénith Nantes Métropole','Stade de la Beaujoire', 'Parc des Expos de la Beaujoire / Exponantes Le Parc','Palais des Sports Beaulieu / H Arena', ]

url = "https://data.nantesmetropole.fr/api/explore/v2.1/catalog/datasets/244400404_agenda-evenements-nantes-nantes-metropole/records?where=date%20%3E%20'2025-02-15'%20AND%20(lieu%20=%20'Cité%20des%20Congrès'%20OR%20lieu%20=%20'STEREOLUX'%20OR%20lieu%20=%20'Zénith%20Nantes%20Métropole'%20OR%20lieu%20=%20'Stade%20de%20la%20Beaujoire'%20OR%20lieu%20=%20'Parc%20des%20Expos%20de%20la%20Beaujoire%20/%20Exponantes%20Le%20Parc'%20OR%20lieu%20=%20'Palais%20des%20Sports%20Beaulieu%20/%20H%20Arena')&order_by=date"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    df_event_v1 = pd.DataFrame(data['results'])
else:
    print("Erreur :", response.status_code, response.text)


In [107]:
# API V2

url = "https://data.nantesmetropole.fr/api/explore/v2.1/catalog/datasets/244400404_agenda-evenements-nantes-metropole_v2/records?where=date%20%3E%20'2025-02-15'%20AND%20(lieu%20=%20'Cité%20des%20Congrès'%20OR%20lieu%20=%20'STEREOLUX'%20OR%20lieu%20=%20'Zénith%20Nantes%20Métropole'%20OR%20lieu%20=%20'Stade%20de%20la%20Beaujoire'%20OR%20lieu%20=%20'Parc%20des%20Expos%20de%20la%20Beaujoire%20/%20Exponantes%20Le%20Parc'%20OR%20lieu%20=%20'Palais%20des%20Sports%20Beaulieu%20/%20H%20Arena')&order_by=date&limit=20"

response = requests.get(url)

if response.status_code == 200:
    data_v2 = response.json()
    df_event_v2 = pd.DataFrame(data_v2['results'])
else:
    print("Erreur :", response.status_code, response.text)


In [108]:
df_concat = pd.concat([df_event_v1,df_event_v2])

In [109]:
df_filterd = df_concat[['lieu','nom','longitude','latitude','type','heure_debut','heure_fin','date']]

In [110]:
df_filterd = df_filterd.drop_duplicates()

In [111]:
len(df_filterd)

30

In [112]:
df_filterd.head()

,lieu,nom,longitude,latitude,type,heure_debut,heure_fin,date
0,Stade de la Beaujoire,Don de sang au Stade de la Beaujoire,-1.524704,47.256019,"Rencontre - Forum,,,,",10:00,13:00,2025-03-19
1,Stade de la Beaujoire,Don de sang au Stade de la Beaujoire,-1.524704,47.256019,"Rencontre - Forum,,,,",15:30,19:30,2025-03-19
2,Cité des Congrès,Julien Lieb,-1.543612,47.212767,"Concert - Musique,,,,",20:30,None,2025-03-21
3,Palais des Sports Beaulieu / H Arena,Handball : HBC Nantes / US Dunkerque Handball,-1.536772,47.209525,"Evènement sportif,,,,",20:00,None,2025-03-22
4,Cité des Congrès,Nora Hamzawi,-1.543612,47.212767,"Théâtre - Humour,,,,",20:00,None,2025-03-22


In [113]:
df = df_filterd[~df_filterd['nom'].str.contains('Don de sang', case=False, na=False)]
df

,lieu,nom,longitude,latitude,type,heure_debut,heure_fin,date
2,Cité des Congrès,Julien Lieb,-1.543612,47.212767,"Concert - Musique,,,,",20:30,None,2025-03-21
3,Palais des Sports Beaulieu / H Arena,Handball : HBC Nantes / US Dunkerque Handball,-1.536772,47.209525,"Evènement sportif,,,,",20:00,None,2025-03-22
4,Cité des Congrès,Nora Hamzawi,-1.543612,47.212767,"Théâtre - Humour,,,,",20:00,None,2025-03-22
5,Cité des Congrès,Festival Vive la Magie,-1.543612,47.212767,"Cirque - Magie - Marionnettes,,,,",18:00,19:45,2025-03-23
6,Cité des Congrès,Festival Vive la Magie,-1.543612,47.212767,"Cirque - Magie - Marionnettes,,,,",15:00,16:45,2025-03-23
7,Cité des Congrès,Thomas Fersen : Vibrations,-1.543612,47.212767,"Concert - Musique,,,,",20:00,None,2025-03-25
8,Cité des Congrès,"Pause-concert : Ravel, Debussy, Berio - Saison ONPL",-1.543612,47.212767,"Concert - Musique,,,,",12:30,13:15,2025-03-25
9,Cité des Congrès,La Mer de Debussy - Saison ONPL,-1.543612,47.212767,"Concert - Musique,,,,",20:30,None,2025-03-25
2,Cité des Congrès,Julien Lieb,-1.543612,47.212767,NaN,20:30,None,2025-03-21
3,Palais des Sports Beaulieu / H Arena,Handball : HBC Nantes / US Dunkerque Handball,-1.536772,47.209525,NaN,20:00,None,2025-03-22


In [114]:
# Scrapping des feuilles de match avec horaires : https://www.fcnantes.com/articles/20242025/feuillematch.php?num=24

In [ ]:
def fetch_datetime_fcmatch(url_feuille_match) :
    url_feuille_match = f'https://www.fcnantes.com/articles/20242025/{url_feuille_match}'
    response_feuille_match = requests.get(url_feuille_match)
    if response_feuille_match.status_code == 200 :
        soup_feuille_martch = BeautifulSoup(response_feuille_match.content,'html.parser')
        time_ = soup_feuille_martch.find('div', class_="principal_fdm").find('center').text.strip() 
        heure_match = re.search(r'\b(\d{1,2}:\d{2})\b', time_)
        if heure_match:
            heure = heure_match.group(1)
            print(heure)
            print(type(heure))
            heures, minutes = map(int, heure.split(':'))
            print(heures)
            print(type(heures))
            print(minutes)
            time_obj = datetime.time(heures, minutes)
            print(time_obj)
            print(type(time_obj))
            return time_obj

In [211]:
# import datetime
# import requests
# from bs4 import BeautifulSoup
# import re

def fetch_datetime_fcmatch(url_feuille_match):
    url_feuille_match = f'https://www.fcnantes.com/articles/20242025/{url_feuille_match}'
    response_feuille_match = requests.get(url_feuille_match)

    if response_feuille_match.status_code == 200:
        soup_feuille_martch = BeautifulSoup(response_feuille_match.content, 'html.parser')
        time_text = soup_feuille_martch.find('div', class_="principal_fdm").find('center').text.strip()

        # Extraction de l'heure avec regex
        heure_match = re.search(r'\b(\d{1,2}):(\d{2})\b', time_text)
        if heure_match:
            heures = int(heure_match.group(1))
            minutes = int(heure_match.group(2))
            print(f"Heure : {heures}, Minutes : {minutes}")

            # Utilise la classe datetime.time proprement
            time_obj = datetime.time(heures, minutes)
            print(f"Time object : {time_obj} / Type : {type(time_obj)}")
            return time_obj
    return None


In [195]:
url_fcnantes = 'https://www.fcnantes.com/articles/20242025/calendrier.php'
page_fc = requests.get(url_fcnantes)
if page_fc.status_code == 200 :
    soup_fcnantes = BeautifulSoup(page_fc.content, 'html.parser')
    match_container = soup_fcnantes.find(id="pageContainer")

In [ ]:
datime_match_list = []
schedule_list = []
date_list = []

matches = match_container.find_all(class_="match")
for match in matches:
    # Correction d'encodage sur les mois
    date_text = match.find(class_="datecal").text.strip().replace('ao√Ľt', 'août').replace('d√©cembre', 'décembre').replace('f√©vrier', 'février')
    
    # Parse propre avec la bonne classe
    date_obj = datetime.datetime.strptime(date_text, "%A %d %B %Y").date()

    # Comparaison avec une date
    if date_obj > datetime.datetime.strptime('2025-02-14', "%Y-%m-%d").date():
        stade = match.find(class_="stade").text.upper().strip()
        
        if 'BEAUJOIRE' in stade:
            score = match.find(class_="scorecal").text.strip()
            blue_btns = match.find_all('div', class_="blue-btn")

            if blue_btns:
                for btn in blue_btns:
                    a_tag = btn.find('a', class_="first-link")
                    if a_tag and "Compo" in a_tag.text:
                        heure = fetch_datetime_fcmatch(a_tag['href'])
                        if heure: 
                            date_time_match = datetime.datetime.combine(date_obj, heure)
                            datime_match_list.append(date_time_match)

            else:
                if score != '- : -':
                    schedule_list.append(score)
                    date_list.append(date_obj)


Heure : 15, Minutes : 0
Time object : 15:00:00 / Type : <class 'datetime.time'>
Heure : 17, Minutes : 15
Time object : 17:15:00 / Type : <class 'datetime.time'>
Heure : 17, Minutes : 0
Time object : 17:00:00 / Type : <class 'datetime.time'>


In [ ]:
#feuille_match_links = []
datime_match_list = []
schedule_list = []
date_list = []

matches = match_container.find_all(class_="match")
for match in matches:
    date = match.find(class_="datecal").text.strip().replace('ao√Ľt', 'août').replace('d√©cembre', 'décembre').replace('f√©vrier', 'février')
    date_obj = datetime.strptime(date, "%A %d %B %Y").date()
    if date_obj > datetime.strptime('2025-02-14', "%Y-%m-%d").date() :
        stade = match.find(class_="stade").text.upper().strip()
        if 'BEAUJOIRE' in stade :
            score = schedule = match.find(class_="scorecal").text.strip()
            blue_btns = match.find_all('div', class_="blue-btn")
            if blue_btns :
                for btn in blue_btns:
                    a_tag = btn.find('a', class_="first-link")
                    if a_tag and "Compo" in a_tag.text:
                        heure = fetch_datetime_fcmatch(a_tag['href'])
                        if heure : 
                            date_time_match  = datetime.combine(date_obj, heure)
                            datime_match_list.append(date_time_match)

            else :
                schedule = score
                if schedule != '- : -' :
                    schedule_list.append(schedule)
                    date_list.append(date_obj)

        else :
            pass


AttributeError: module 'datetime' has no attribute 'strptime'

In [ ]:
url_feuille_match = 'https://www.fcnantes.com/articles/20242025/feuillematch.php?num=1'
response_feuille_match = requests.get(url_feuille_match)
if response_feuille_match.status_code == 200 :
    soup_feuille_martch = BeautifulSoup(response_feuille_match.content,'html.parser')
    time_ = soup_feuille_martch.find('div', class_="principal_fdm").find('center').text.strip() 
    heure_match = re.search(r'\b(\d{1,2}:\d{2})\b', time_)
    if heure_match:
        heure = heure_match.group(1)

🕒 Heure trouvée : 17:00


# H Arena

In [115]:
# scrappinng H : https://hbcnantes.com/equipe-professionnelle/calendrier-et-resultats/

In [3]:
def get_Harena_datetime(calendar_tag) :
    date_text = calendar_tag.get_text(separator=' ').strip()
    clean_text = re.sub(r'\s+', ' ', date_text).strip()
    # Dictionnaire des mois français
    mois_fr = {
        'janv.': '01', 'févr.': '02', 'mars': '03', 'avr.': '04',
        'mai': '05', 'juin': '06', 'juil.': '07', 'août': '08',
        'sept.': '09', 'oct.': '10', 'nov.': '11', 'déc.': '12'
    }

    # Split des parties
    parts = clean_text.split()
    jour = parts[0]
    mois = mois_fr[parts[1]]
    annee = parts[2]
    heure = parts[3]

    # Reformatage en string de type "31-08-2024 20:00"
    date_str = f"{jour}-{mois}-{annee} {heure}"

    # Conversion en datetime
    dt = datetime.strptime(date_str, "%d-%m-%Y %H:%M")
    return dt  

In [ ]:
date_time_list = []
date_reference = datetime.strptime('2025-02-15 20:00', "%Y-%m-%d %H:%M")

url_h = 'https://hbcnantes.com/equipe-professionnelle/calendrier-et-resultats/'
page = requests.get(url_h)
if page.status_code == 200 :
    soup = BeautifulSoup(page.content, 'html.parser')
    event_list = soup.find('div', class_="calendar").find('div', class_="container")
    calendar_item = event_list.find_all('div',class_='row calendar__row align-items-center no-gutters mb-2')
    
    for item in calendar_item :
        home_team = item.find('div', class_="calendar__row-teamname").text.strip()
        if home_team == 'HBC Nantes' :
            calendar_tag = item.find('div', class_="d-flex align-items-center").find('div', class_="calendar__row-date")
            datime_match = get_Harena_datetime(calendar_tag)
            if (datime_match > date_reference) and (datime_match.time() != time(0, 0, 0)):
                date_time_list.append(datime_match)
df_H_arena = pd.DataFrame({
    'lieu' : 'Palais des Sports Beaulieu / H Arena',
    'nom' : 'Match handball',
    'longitude' : '-1.536772',
    'latitude' : '47.209525',
    'type' : 'Evènement sportif',
    'heure_debut': [dt.strftime('%H:%M') if dt else None for dt in date_time_list], 
    'heure_fin' :[dt.strftime('%H:%M') if dt else None for dt in date_time_list],
    'date' : [time.date() for time in date_time_list], 
    'datetime' : date_time_list,
})

df_H_arena.head()

,Evenement,Datetime,Date_debut,Heure_debut,Heure_fin
0,Match handball,2025-02-19 20:45:00,2025-02-19,20:45:00,20:45:00
1,Match handball,2025-02-22 20:30:00,2025-02-22,20:30:00,20:30:00
2,Match handball,2025-03-01 20:30:00,2025-03-01,20:30:00,20:30:00
3,Match handball,2025-03-05 20:45:00,2025-03-05,20:45:00,20:45:00
4,Match handball,2025-03-22 20:00:00,2025-03-22,20:00:00,20:00:00


In [116]:
# Stereolux et cite des congres : https://openagenda.com/fr/nantesmetropole?timings%5Bgte%5D=2025-02-14T23%3A00%3A00.000Z&timings%5Blte%5D=2025-08-31T21%3A59%3A59.999Z&timings%5Btz%5D=Europe%2FParis&locationUid%5B0%5D=47538434&locationUid%5B1%5D=21547859

In [ ]:

# Configure le WebDriver en auto-download
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)


url = "https://openagenda.com/fr/nantesmetropole?locationUid%5B0%5D=47538434&locationUid%5B1%5D=21547859&timings%5Bgte%5D=2025-02-14T23%3A00%3A00.000Z&timings%5Blte%5D=2025-03-24T22%3A59%3A59.999Z&timings%5Btz%5D=Europe%2FParis"
driver.get(url)

# Scroller pour charger tout le contenu
scroll_pause = 2
last_height = driver.execute_script("return document.body.scrollHeight")

for _ in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

driver.quit()


In [32]:
event_tag = soup.find('div', class_="oa-1o8sgt3").find('div', class_="oa-11ydnqy")
event_list = event_tag.find_all('article', class_="oa-z3hrkb")

In [57]:
from datetime import datetime
import re
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')


def extraire_datetimes(texte, annee=2025):
    """
    Extrait toutes les datetimes d'une chaîne du type :
    'Samedi 15 février, 17h30, 20h30, 22h00'
    et retourne une liste de datetime
    """
    # Récupère la partie date (ex: '15 février')
    match_date = re.search(r'(\d{1,2} \w+)', texte)
    if not match_date:
        return []  # Pas de date trouvée
    
    date_str = match_date.group(1)
    # Extrait toutes les heures type 17h30
    heures = re.findall(r'(\d{1,2}h\d{2})', texte)

    datetimes = []
    for heure in heures:
        full_str = f"{date_str} {annee} {heure}"
        try:
            dt = datetime.strptime(full_str, "%d %B %Y %Hh%M")
            datetimes.append(dt)
        except ValueError:
            pass  
    return datetimes


In [102]:
place_list = []
datetime_list = []
name_list = []

for event in event_list :
    info_section = event.find('section')
    date_time_section = info_section.find('div', class_="oa-gg4vpm").find_all('li')
    name = info_section.find('div', class_="oa-1bzcof").text.strip().replace('Complet', '')
    date_time = date_time_section[0].text
    place = date_time_section[1].text
    
    if 'Congrès' in place :
        place = 'Cité des Congrès'
    elif 'STEREOLUX' in place :
        place = 'STEREOLUX'
    
    datetimes = extraire_datetimes(date_time)
    if len(datetimes) > 1 :
        amount_datetimes = len(datetimes)
        [datetime_list.append(datetime_) for datetime_ in datetimes]
        [place_list.append(place) for i in range(len(datetimes))]
        [name_list.append(name) for i in range(len(datetimes))]
        
    else :
        place_list.append(place)
        datetime_list.append(datetimes[0] if datetimes else None)
        name_list.append(name)
        
if len(place_list) == len(datetime_list) == len(name_list) :       
    df_stereolux_congres = pd.DataFrame({
        'lieu' : place_list,
        'nom' : name_list,
        'longitude' : '',
        'latitude' : '',
        'type' : 'Culture',
        'heure_debut': [dt.strftime('%H:%M') if dt else None for dt in datetime_list], 
        'heure_fin' : [dt.strftime('%H:%M') if dt else None for dt in datetime_list],
        'date' : [dt.date() if dt else None for dt in datetime_list], 
        'datetime' : datetime_list
    })
else :
    raise Exception('all list must be the same lenght')

In [103]:
df_stereolux_congres['latitude'] = df_stereolux_congres.apply(lambda x : '47.212767' if x['lieu'] == 'Cité des Congrès' else '47.206315', axis=1)
df_stereolux_congres['longitude'] = df_stereolux_congres.apply(lambda x : '-1.543612' if x['lieu'] == 'Cité des Congrès' else '-1.564217', axis=1)

In [104]:
df_stereolux_congres.head()

,lieu,nom,longitude,latitude,type,heure_debut,heure_fin,date,datetime
0,Cité des Congrès,"Les hommes viennent de Mars, les femmes de Vén...",-1.543612,47.212767,Culture,17:30,17:30,2025-02-15,2025-02-15 17:30:00
1,Cité des Congrès,"Les hommes viennent de Mars, les femmes de Vén...",-1.543612,47.212767,Culture,20:30,20:30,2025-02-15,2025-02-15 20:30:00
2,Cité des Congrès,Tout va très bien ! - avec Laurent Ournac et A...,-1.543612,47.212767,Culture,17:00,17:00,2025-02-16,2025-02-16 17:00:00
3,STEREOLUX,The Libertines,-1.564217,47.206315,Culture,18:30,18:30,2025-02-16,2025-02-16 18:30:00
4,Cité des Congrès,Simon Superlapin : Superlapin et la pierre qui...,-1.543612,47.212767,Culture,14:00,14:00,2025-02-23,2025-02-23 14:00:00


In [ ]:
url_stereolux_congres = 'https://openagenda.com/fr/nantesmetropole?locationUid%5B0%5D=47538434&locationUid%5B1%5D=21547859&timings%5Bgte%5D=2025-02-14T23%3A00%3A00.000Z&timings%5Blte%5D=2025-03-24T22%3A59%3A59.999Z&timings%5Btz%5D=Europe%2FParis'
page_agenda = requests.get(url_stereolux_congres)
if page_agenda.status_code == 200  :
    soup_agenda = BeautifulSoup(page_agenda.content, 'html.parser')
    

<!DOCTYPE html>
<html data-theme="light" lang="fr" style="color-scheme:light"><head><meta charset="utf-8" data-next-head=""/><meta content="width=device-width, initial-scale=1, maximum-scale=5, minimum-scale=1, viewport-fit=cover" data-next-head="" name="viewport"/><meta content="Communicate efficiently on your events" data-next-head="" name="description"/><meta content="openagenda, agendas, events, opendata, open data, network, networked" data-next-head="" name="keywords"/><meta content="#41ACDD" data-next-head="" name="theme-color"/><title data-next-head="">Nantes Métropole | OpenAgenda</title><meta content="index, follow" data-next-head="" name="robots"/><link data-next-head="" href="https://openagenda.com/fr/nantesmetropole" rel="canonical"/><link data-next-head="" href="https://openagenda.com/en/nantesmetropole" hreflang="en" rel="alternate"/><link data-next-head="" href="https://openagenda.com/fr/nantesmetropole" hreflang="fr" rel="alternate"/><link data-next-head="" href="https:

In [117]:
import requests
import json

# URL de base de l'API
BASE_URL = "https://data.nantesmetropole.fr/api/explore/v2.1/catalog/datasets/244400404_agenda-evenements-nantes-metropole_v2/records"

# Requête combinée avec WHERE et OR pour plusieurs lieux + filtre de date
params = {
    "where": "date>'2025-02-15' AND ("
             "lieu='Cité des Congrès' OR "
             "lieu='STEREOLUX' OR "
             "lieu='Zénith Nantes Métropole' OR "
             "lieu='Stade de la Beaujoire' OR "
             "lieu='Parc des Expos de la Beaujoire / Exponantes Le Parc' OR "
             "lieu='Palais des Sports Beaulieu / H Arena')"
}

# Appel API
response = requests.get(BASE_URL, params=params)

# Vérification de la réponse
if response.status_code == 200:
    data = response.json()
    print(f"Nombre d'événements trouvés : {len(data.get('results', []))}")
    df = pd.DataFrame(data['results'])
    print(df.head())
    print(len(df))

#     # Affichage des 5 premiers événements pour exemple
#     for event in data.get('results', [])[:5]:
#         print(f"- {event.get('titre', 'Sans titre')} le {event.get('date', 'Date inconnue')} à {event.get('lieu', 'Lieu inconnu')}")
# else:
#     print(f"Erreur API : {response.status_code} - {response.text}")


Nombre d'événements trouvés : 10
  id_manif  id_agenda_in                                       nom  \
0    68424       6270274                     Les Pixies en concert   
1    68297       6272196                                    Treaks   
2    67912       6262365                           Foire de Nantes   
3    68423       6270272       Le Zouzou Tour de Philippe Katerine   
4    67761       6260220  Handball : HBC Nantes / US Ivry Handball   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                   description_evt  \
0                                                                           

In [120]:
import requests
import pandas as pd

# Liste des lieux à interroger
lieux = [
    "Cité des Congrès",
    "STEREOLUX",
    "Zénith Nantes Métropole",
    "Stade de la Beaujoire",
    "Parc des Expos de la Beaujoire / Exponantes Le Parc",
    "Palais des Sports Beaulieu / H Arena"
]

# URL de base de l'API
BASE_URL = "https://data.nantesmetropole.fr/api/explore/v2.1/catalog/datasets/244400404_agenda-evenements-nantes-metropole_v2/records"

# Date de début de la recherche
DATE_DEBUT = "2025-02-15"

# Liste pour stocker tous les événements
all_events = []

# Boucle sur chaque lieu
for lieu in lieux:
    params = {
        "where": f"date>'{DATE_DEBUT}' AND lieu='{lieu}'"
        }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        events = data.get('results', [])
        print(f"{len(events)} événements trouvés pour {lieu}")
        all_events.extend(events)  # On ajoute les événements à la liste globale
    else:
        print(f"Erreur pour {lieu} : {response.status_code} - {response.text}")

print(f"\nNombre total d'événements collectés : {len(all_events)}")

# ✅ Transformation en DataFrame Pandas
df = pd.DataFrame(all_events)

# ✅ Sélection des colonnes demandées (certaines peuvent être manquantes pour certains événements)
colonnes_finales = ['lieu', 'nom', 'longitude', 'latitude', 'heure_debut', 'heure_fin', 'date']
df_final = df[colonnes_finales]

# ✅ Affichage des premières lignes pour vérification
print("\nAperçu des données :")
print(df_final.head())

# ✅ Export en CSV (optionnel)
df_final.to_csv('evenements_nantes_lieux_specifiques.csv', index=False, encoding='utf-8')
print("\n✅ Données exportées vers 'evenements_nantes_lieux_specifiques.csv'")


10 événements trouvés pour Cité des Congrès
10 événements trouvés pour STEREOLUX
8 événements trouvés pour Zénith Nantes Métropole
6 événements trouvés pour Stade de la Beaujoire
5 événements trouvés pour Parc des Expos de la Beaujoire / Exponantes Le Parc
5 événements trouvés pour Palais des Sports Beaulieu / H Arena

Nombre total d'événements collectés : 44

Aperçu des données :
               lieu  \
0  Cité des Congrès   
1  Cité des Congrès   
2  Cité des Congrès   
3  Cité des Congrès   
4  Cité des Congrès   

                                                                               nom  \
0                                            Neko Light Orchestra : Echos d’Hyrule   
1                                                  Jean-Paul Rouve : L’air de rien   
2                                                     D'jal : En pleine conscience   
3                                         Olivier de Benoist : Le droit au bonheur   
4  Aterballetto - Centre Chorégraphique Nationa

In [123]:
df_final.sort_values('date')

,lieu,nom,longitude,latitude,heure_debut,heure_fin,date
28,Stade de la Beaujoire,Don de sang au Stade de la Beaujoire,-1.524704,47.256019,15:30,19:30,2025-03-19
31,Stade de la Beaujoire,Don de sang au Stade de la Beaujoire,-1.524704,47.256019,10:00,13:00,2025-03-19
6,Cité des Congrès,Julien Lieb,-1.543612,47.212767,20:30,None,2025-03-21
9,Cité des Congrès,Nora Hamzawi,-1.543612,47.212767,20:00,None,2025-03-22
40,Palais des Sports Beaulieu / H Arena,Handball : HBC Nantes / US Dunkerque Handball,-1.536772,47.209525,20:00,None,2025-03-22
7,Cité des Congrès,"Pause-concert : Ravel, Debussy, Berio - Saison ONPL",-1.543612,47.212767,12:30,13:15,2025-03-25
25,Zénith Nantes Métropole,Jarry : Bonhomme,-1.627992,47.228697,20:00,None,2025-03-27
19,STEREOLUX,Miossec,-1.564217,47.206315,20:00,23:00,2025-03-28
4,Cité des Congrès,Aterballetto - Centre Chorégraphique National italien - Saison Fleuriaye nomade,-1.543612,47.212767,20:45,22:00,2025-04-03
37,Parc des Expos de la Beaujoire / Exponantes Le Parc,Foire de Nantes,-1.530469,47.259046,10:00,19:00,2025-04-03


In [125]:
import requests
import pandas as pd

# Liste des lieux à interroger
lieux = [
    "Cité des Congrès",
    "STEREOLUX",
    "Zénith Nantes Métropole",
    "Stade de la Beaujoire",
    "Parc des Expos de la Beaujoire / Exponantes Le Parc",
    "Palais des Sports Beaulieu / H Arena"
]

# URL de base de l'API
BASE_URL = "https://data.nantesmetropole.fr/api/explore/v2.1/catalog/datasets/244400404_agenda-evenements-nantes-nantes-metropole/records"

# Date de début de la recherche
DATE_DEBUT = "2025-02-15"

# Liste pour stocker tous les événements
all_events = []

# Boucle sur chaque lieu
for lieu in lieux:
    params = {
        "where": f"date>'{DATE_DEBUT}' AND lieu='{lieu}'"
        }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        events = data.get('results', [])
        print(f"{len(events)} événements trouvés pour {lieu}")
        all_events.extend(events)  # On ajoute les événements à la liste globale
    else:
        print(f"Erreur pour {lieu} : {response.status_code} - {response.text}")

print(f"\nNombre total d'événements collectés : {len(all_events)}")

# ✅ Transformation en DataFrame Pandas
df = pd.DataFrame(all_events)

# ✅ Sélection des colonnes demandées (certaines peuvent être manquantes pour certains événements)
colonnes_finales = ['lieu', 'nom', 'longitude', 'latitude', 'heure_debut', 'heure_fin', 'date']
df_final = df[colonnes_finales]

# ✅ Affichage des premières lignes pour vérification
print("\nAperçu des données :")
print(df_final.head())

# ✅ Export en CSV (optionnel)
df_final.to_csv('evenements_nantes_lieux_specifiquesV1.csv', index=False, encoding='utf-8')
print("\n✅ Données exportées vers 'evenements_nantes_lieux_specifiques.csv'")


10 événements trouvés pour Cité des Congrès
0 événements trouvés pour STEREOLUX
8 événements trouvés pour Zénith Nantes Métropole
6 événements trouvés pour Stade de la Beaujoire
5 événements trouvés pour Parc des Expos de la Beaujoire / Exponantes Le Parc
5 événements trouvés pour Palais des Sports Beaulieu / H Arena

Nombre total d'événements collectés : 34

Aperçu des données :
               lieu                                 nom  longitude   latitude  \
0  Cité des Congrès             En bohême - Saison ONPL  -1.543612  47.212767   
1  Cité des Congrès                         Julien Lieb  -1.543612  47.212767   
2  Cité des Congrès              Les comédies musicales  -1.543612  47.212767   
3  Cité des Congrès  Enrico Macias : La fête continue !  -1.543612  47.212767   
4  Cité des Congrès                      Verino : Rodéo  -1.543612  47.212767   

  heure_debut heure_fin        date  
0       20:00     21:20  2025-04-22  
1       20:30      None  2025-03-21  
2       20:00   

In [126]:
df_final

,lieu,nom,longitude,latitude,heure_debut,heure_fin,date
0,Cité des Congrès,En bohême - Saison ONPL,-1.543612,47.212767,20:00,21:20,2025-04-22
1,Cité des Congrès,Julien Lieb,-1.543612,47.212767,20:30,None,2025-03-21
2,Cité des Congrès,Les comédies musicales,-1.543612,47.212767,20:00,None,2025-04-05
3,Cité des Congrès,Enrico Macias : La fête continue !,-1.543612,47.212767,20:30,None,2025-05-24
4,Cité des Congrès,Verino : Rodéo,-1.543612,47.212767,20:00,None,2025-06-06
5,Cité des Congrès,Laurent Baffie se pose des questions,-1.543612,47.212767,20:00,None,2025-06-12
6,Cité des Congrès,Neko Light Orchestra : Échos de la terre du milieu et de Westeros,-1.543612,47.212767,20:00,None,2025-04-13
7,Cité des Congrès,Tom Baldetti – Tome 1,-1.543612,47.212767,20:30,None,2025-04-24
8,Cité des Congrès,Azur et Asmar - Ciné-concert,-1.543612,47.212767,20:00,None,2025-05-13
9,Cité des Congrès,"Thomas Marty : Allez, la bise !",-1.543612,47.212767,17:00,None,2025-05-18


In [131]:
# cite des congres
url = 'https://data.nantesmetropole.fr/api/explore/v2.1/catalog/datasets/244400404_agenda-evenements-nantes-nantes-metropole/records?order_by=date&refine=lieu%3A%22Cit%C3%A9%20des%20Congr%C3%A8s%22&limit=100'
response = requests.get(url)

if response.status_code == 200:
    data_v2 = response.json()
    df_event_v2 = pd.DataFrame(data_v2['results'])
else:
    print("Erreur :", response.status_code, response.text)


In [132]:
colonnes_finales = ['lieu', 'nom', 'longitude', 'latitude', 'heure_debut', 'heure_fin', 'date']
df_final = df_event_v2[colonnes_finales]
df_final

,lieu,nom,longitude,latitude,heure_debut,heure_fin,date
0,Cité des Congrès,Conseil métropolitain,-1.543612,47.212767,09:00,None,2025-02-07
1,Cité des Congrès,Julien Lieb,-1.543612,47.212767,20:30,None,2025-03-21
2,Cité des Congrès,Nora Hamzawi,-1.543612,47.212767,20:00,None,2025-03-22
3,Cité des Congrès,Festival Vive la Magie,-1.543612,47.212767,18:00,19:45,2025-03-23
4,Cité des Congrès,Festival Vive la Magie,-1.543612,47.212767,15:00,16:45,2025-03-23
5,Cité des Congrès,Thomas Fersen : Vibrations,-1.543612,47.212767,20:00,None,2025-03-25
6,Cité des Congrès,"Pause-concert : Ravel, Debussy, Berio - Saison ONPL",-1.543612,47.212767,12:30,13:15,2025-03-25
7,Cité des Congrès,La Mer de Debussy - Saison ONPL,-1.543612,47.212767,20:30,None,2025-03-25
8,Cité des Congrès,Les Franglaises,-1.543612,47.212767,20:00,None,2025-03-27
9,Cité des Congrès,Les Franglaises,-1.543612,47.212767,20:00,None,2025-03-28
